In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/lga.csv')

In [4]:
dataset.shape

(8757, 1045)

In [5]:
dataset.head(3)

,Date,Hour,1,10,100,101,102,106,107,108,...,91_lag_3,92_lag_3,93_lag_3,94_lag_3,95_lag_3,96_lag_3,97_lag_3,98_lag_3,99_lag_3,arrival_lag_3
0,2018-01-01,3,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
1,2018-01-01,4,1,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2018-01-01,5,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

774

In [7]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 271)

In [8]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

257

In [9]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [10]:
sep = int(0.75*len(dataset))
print(sep)


trainData = dataset[:sep]
testData = dataset[sep:]

print(trainData.shape)
print(testData.shape)

6567
(6567, 271)
(2190, 271)


In [11]:
X_train = trainData[features_cols].values
X_train = torch.tensor(X_train).float().to(device)
print(X_train.shape)

y_train = trainData[targetColumns].values
y_train = torch.tensor(y_train).float().to(device)
print(y_train.shape)

X_test = testData[features_cols].values
X_test = torch.tensor(X_test).float().to(device)
print(X_test.shape)

y_test = testData[targetColumns].values
y_test = torch.tensor(y_test).float().to(device)
print(y_test.shape)

torch.Size([6567, 13])
torch.Size([6567, 257])
torch.Size([2190, 13])
torch.Size([2190, 257])


In [12]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
        train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [13]:
bptt = 6

In [14]:
train_inout_seq = create_inout_sequences(X_train,y_train, bptt)

In [15]:
train_inout_seq[0][0].shape,train_inout_seq[0][1].shape

(torch.Size([6, 270]), torch.Size([6, 257]))

In [16]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1, layers=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size, num_layers=layers)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(layers,1,self.hidden_layer_size),
                            torch.zeros(layers,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
#         return predictions[-1]
        return predictions

In [17]:
layers = 2
model = LSTM(input_size = 270, hidden_layer_size=100, output_size=257, layers=layers).to(device)
loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:
epochs = 200


for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
                        torch.zeros(layers, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 1.27257872
epoch:  26 loss: 1.21691644
epoch:  51 loss: 1.19304931
epoch:  76 loss: 1.18942380
epoch: 101 loss: 1.19723511
epoch: 126 loss: 1.19722164
epoch: 151 loss: 1.30239308
epoch: 176 loss: 1.20324516
epoch: 199 loss: 1.1983643770


In [19]:
test_inout_seq = create_inout_sequences(X_test,y_test, bptt)

In [20]:
model.eval()
prediction = []
with torch.no_grad():
    for seq, labels in test_inout_seq:
        model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
                        torch.zeros(layers, 1, model.hidden_layer_size))
        prediction.append(model(seq)[-1])

In [21]:
y_test_ = torch.stack([labels[-1] for seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
y_pred_ = torch.stack(prediction).detach().cpu().numpy()

In [22]:
y_test_.shape, y_pred_.shape

((2184, 257), (2184, 257))

In [23]:
r2_score(y_test_, y_pred_, multioutput='variance_weighted')

0.7235033029491722

In [24]:
# 300 epoch MSE bptt 12 hidden=100
#0.4597512505107641

In [25]:
# 150 epoch MSE bptt 12 hidden=100
# 0.47003788967204657

In [26]:
# 150 epoch L1 bptt 12 hidden=100
# 0.49459994974800275

In [27]:
# 300 epoch L1 bptt 12 hidden=100
# 0.48786324620679394

In [28]:
# 150 epoch L1 bptt 6 hidden=100
# 0.5141071457595932

In [29]:
# 150 epoch L1 bptt=6 hidden=50 
# 0.4721109077014878

In [30]:
# 150 epoch L1 bptt=6 hidden=200 
# 0.4574538111782432

In [31]:
# 300 epoch L1 bptt 6 hidden=100
# 0.5001768915084189

In [32]:
# 150 epoch L1 bptt 6 hidden=100 layers 2
# 0.5157145438811139

In [33]:
# 150 epoch L1 bptt 6 hidden=100 layers 3
# 0.5072893536554984